In [12]:
import pandas as pd
import pickle

with open('p5r_data.pkl', 'rb') as file:
    data = pickle.load(file)

df_persona = pd.read_csv('p5r_人格面具.csv', index_col=0)
df_compose = pd.read_csv('p5r_合成总表.csv')
df_item = pd.read_csv('p5r_道具图鉴.csv', index_col=0)

persona_level = dict(zip(df_persona['中文名'].tolist(), df_persona['等级'].tolist()))
df_compose['Level'] = df_compose['Result'].map(persona_level)
df_compose['Lv.1'] = df_compose['Name.1'].map(persona_level).fillna(df_compose['Lv.1']).fillna(-1).astype(int)

df_item['备注'] = df_item['备注'].str.split('/').str[-1]
df_item['Lv'] = df_item['备注'].map(persona_level).fillna(-1).astype(int)

df_compose.head()

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
0,6561.0,愚者,8,背负怪,愚者,11,俄耳普斯F,亚森,1
1,7077.0,愚者,8,背负怪,愚者,13,俄耳普斯·贼神F,亚森,1
2,6084.0,魔术师,5,猫妖精,魔术师,11,杰克霜精,皮克希,2
3,6084.0,魔术师,5,猫妖精,魔术师,11,杰克霜精,皮克希,2
4,6741.0,恋人,6,幸魂,恋人,13,天钿女命,皮克希,2


In [17]:
# 查询反向合成表
df_compose.loc[df_compose['Result'].str.contains('吉祥天')]

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
17074,78660.0,信念,65,大元帅明王,审判,59,吹号者,吉祥天,69
17075,79956.0,顾问官,71,狄俄尼索斯,太阳,53,象头神,吉祥天,69
17076,81714.0,月,70,拜亚基,女皇,56,蒂坦妮娅,吉祥天,69
17077,82041.0,恶魔,74,奈比洛斯,倒悬者,51,佳塔由,吉祥天,69
17078,82377.0,审判,59,吹号者,星星,68,婆苏吉,吉祥天,69
...,...,...,...,...,...,...,...,...,...
17258,157620.0,信念,93,马利亚,审判,87,米迦勒,吉祥天,69
17259,162525.0,审判,90,弥赛亚·贼神,星星,93,路西法,吉祥天,69
17260,162525.0,信念,93,马利亚,审判,90,弥赛亚·贼神,吉祥天,69
17261,165885.0,审判,92,撒旦,星星,93,路西法,吉祥天,69


In [7]:
# 查询技能，可尝试多个idx
name = '杰克霜精'
print(f'可选表：{list(range(len(data[name])))}')
data[name][2].iloc[4:, 1:6]

可选表：[0, 1, 2, 3, 4]


,1,2,3,4,5
4,技能,消耗,解锁等级,解锁等级,描述
5,布芙,4 SP,自带,自带,对1名敌人造成冰冻属性的小量伤害，罕见使敌人陷入冻结
6,消除冰冻防御,6 SP,自带,自带,消除所有敌人的冰冻耐性，无法消除防冰之壁
7,拜斯堤,4 SP,自带,自带,治疗1名伙伴的燃烧、冻结、触电状态
8,玛哈布芙,10 SP,12,12,对所有敌人造成冰冻属性的小量伤害，罕见使敌人陷入冻结
9,拉坤达,8 SP,13,13,在3回合间，使1名敌人的防御力下降
10,冻结几率UP,NaN,15,15,NaN


In [18]:
# 查询任意种族，等级面具
df_persona.loc[df_persona['中文名'].str.contains('迦楼罗')]
# df_persona.loc[df_persona['等级'] == 64]
# df_persona.loc[df_persona['塔罗牌'] == '审判']

,中文名,等级,英文名,日文名,塔罗牌,物,枪,火,冰,电,风,念,核,祝,咒
147,迦楼罗,52,Garuda,ガルーダ,星星,-,-,-,-,-,耐,-,-,反,-


In [16]:
# 指定等级范围查询一步能合成的面具
name = '地狱天使'
level = "1 - 100"

level_range = level.split(' - ')
name_cond = (df_compose['Name'].str.contains(name)) | (df_compose['Name.1'].str.contains(name))
level_cond = (df_compose['Level'] >= int(level_range[0])) & (df_compose['Level'] < int(level_range[1]))
df_compose.loc[name_cond & level_cond]
# df_compose.loc[name_cond & level_cond, ['Result', 'Level']].drop_duplicates('Result')     # 仅显示合成结果及等级

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
1275,18024.0,死神,37,地狱天使,愚者,1,亚森,鬼,20
1309,18180.0,死神,3,曼陀罗,死神,37,地狱天使,奴延,20
1310,19080.0,死神,9,恶灵,死神,37,地狱天使,奴延,20
1311,21288.0,死神,17,斗牛士,死神,37,地狱天使,奴延,20
1312,26205.0,死神,28,毕舍遮,死神,37,地狱天使,奴延,20
...,...,...,...,...,...,...,...,...,...
19089,99720.0,信念,93,马利亚,死神,37,地狱天使,劳尔,76
19153,73149.0,太阳,76,阿修罗王,死神,37,地狱天使,斯卡哈,77
19454,76005.0,死神,37,地狱天使,恋人,78,拉斐尔,加百列,77
19531,86568.0,死神,37,地狱天使,恋人,85,伊丝塔,加百列,77


In [10]:
from search_route import search_route

start = '默基瑟德'  # 设置合成起点
end = '爱丽丝'      # 设置合成终点
head_num = 10       # 设置显示个数（按合成成本升序）
max_depth = 3       # 设置搜索最大深度

search_route(start, end, max_depth=max_depth, head_num=head_num)

爱丽丝 特殊合成方式:
    种族  Lv  Name
1  恶魔  82   贝利亚
2  恶魔  74  奈比洛斯 

默基瑟德 + 莉莉丝 -> 奈比洛斯 = 71730.0 -特殊合成-> 爱丽丝
默基瑟德 + 拜亚基 -> 奈比洛斯 = 83850.0 -特殊合成-> 爱丽丝
默基瑟德 + 圣达芬 -> 奈比洛斯 = 90585.0 -特殊合成-> 爱丽丝
默基瑟德 + 希路奇 -> 苍白骑士 + 托尔 -> 奈比洛斯 = 109548.0 -特殊合成-> 爱丽丝
默基瑟德 + 飞天 -> 苍白骑士 + 托尔 -> 奈比洛斯 = 110523.0 -特殊合成-> 爱丽丝
默基瑟德 + 天钿女命 -> 八岐大蛇 + 八咫乌 -> 奈比洛斯 = 114162.0 -特殊合成-> 爱丽丝
默基瑟德 + 凯尔皮 -> 阿难陀 + 阿巴顿 -> 奈比洛斯 = 114531.0 -特殊合成-> 爱丽丝
默基瑟德 + 比利士 -> 百臂巨人 + 库夫林 -> 奈比洛斯 = 114831.0 -特殊合成-> 爱丽丝
默基瑟德 + 塔姆林 -> 佳塔由 + 大元帅明王 -> 奈比洛斯 = 114909.0 -特殊合成-> 爱丽丝
默基瑟德 + 塔姆林 -> 佳塔由 + 奥伯隆 -> 奈比洛斯 = 116130.0 -特殊合成-> 爱丽丝


In [13]:
# 查询道具图鉴

cond_app = df_item['适用'].str.contains('龙司')
cond_type = df_item['类型'].str.contains('近战武器')
cond_all = True
df_item.loc[cond_type & cond_app]

,中文名,日文名,类型,属性,追加效果,获取,适用,备注,Lv
100,战锤,バトルハンマー,近战武器,攻94 命93,NaN,商店售卖,坂本龙司,5月18,-1
105,重钢管,重鋼管,近战武器,攻76 命91,低机率附加混乱,商店售卖,坂本龙司,5月18,-1
109,地狱强棒,ヘルスラッガー,近战武器,攻66 命93,NaN,商店售卖,坂本龙司,4月17,-1
113,铁管,鉄パイプ,近战武器,攻56 命91,力+1,商店售卖,坂本龙司,4月17,-1
146,扼杀的妙尔尼尔,NaN,近战武器,攻324 命88,电击攻击的威力增加,警报电刑,坂本龙司,托尔,64
149,最终之锤,ラストハンマー,近战武器,攻320 命93,NaN,商店售卖,坂本龙司,1月13,-1
165,始祖钉锤,アークメイス,近战武器,攻300 命91,暴击率小UP,商店售卖,坂本龙司,1月13,-1
173,破天如意棒,NaN,近战武器,攻290 命88,在战斗开始时发动塔尔卡加,警报电刑,坂本龙司,哈奴曼,64
176,超百万吨棍棒,超メガトンロッド,近战武器,攻284 命93,NaN,商店售卖,坂本龙司,12月22,-1
181,妙尔尼尔,NaN,近战武器,攻280 命88,电击攻击的威力增加,警报电刑,坂本龙司,托尔,64
